In [1]:
import os
import numpy as np
import pandas as pd
from autogluon.tabular import TabularPredictor
import warnings
warnings.filterwarnings('ignore')

# ============================================================
# 0) Load
# ============================================================
TRAIN_PATH = "../data/train.csv"
TEST_PATH  = "../data/test.csv"
SUB_PATH   = "../data/sample_submission.csv"

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SUB_PATH)

TARGET_COL = "임신 성공 여부"
ID_COL = "ID"

# ============================================================
# 1) Feature Engineering (검증된 20개만!)
# ============================================================

# 고결측 변수 제거
HIGH_MISSING_COLS = [
    '난자 해동 경과일',           # 99.4%
    'PGS 시술 여부',             # 99.2%
    'PGD 시술 여부',             # 99.1%
    '착상 전 유전 검사 사용 여부', # 98.9%
    '임신 시도 또는 마지막 임신 경과 연수',  # 96.3%
]

EMBRYO_STAGE_COLS = [
    "단일 배아 이식 여부","착상 전 유전 진단 사용 여부","배아 생성 주요 이유",
    "총 생성 배아 수","미세주입된 난자 수","미세주입에서 생성된 배아 수",
    "이식된 배아 수","미세주입 배아 이식 수","저장된 배아 수",
    "미세주입 후 저장된 배아 수","해동된 배아 수","해동 난자 수",
    "수집된 신선 난자 수","저장된 신선 난자 수","혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수","기증자 정자와 혼합된 난자 수",
    "동결 배아 사용 여부","신선 배아 사용 여부","기증 배아 사용 여부","대리모 여부",
]

INFERTILITY_COLS = [
    "남성 주 불임 원인","남성 부 불임 원인","여성 주 불임 원인","여성 부 불임 원인",
    "부부 주 불임 원인","부부 부 불임 원인","불명확 불임 원인",
    "불임 원인 - 난관 질환","불임 원인 - 남성 요인","불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인","불임 원인 - 자궁경부 문제","불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도","불임 원인 - 정자 면역학적 요인","불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태"
]

def safe_div(a, b):
    return np.where(b == 0, 0.0, a / b)

def preprocess(df):
    df = df.copy()
    
    # 고결측 제거
    df = df.drop(columns=HIGH_MISSING_COLS, errors='ignore')
    
    # ===== 03번 검증된 Feature (15개) =====
    
    # 1. 시술_대분류
    def major_procedure(x):
        if pd.isna(x): return "Unknown"
        x = str(x)
        if "IUI" in x: return "IUI"
        if "ICSI" in x: return "ICSI"
        if "IVF" in x: return "IVF"
        if "DI" in x: return "DI"
        return "Other"
    
    df["시술_대분류"] = df["특정 시술 유형"].apply(major_procedure)
    
    # 2. BLASTOCYST_포함
    s = df["특정 시술 유형"].astype("object").fillna("Unknown").astype(str)
    df["BLASTOCYST_포함"] = s.str.contains("BLASTOCYST", na=False).astype(int)
    
    # 3. 배아_이식_여부
    df["배아_stage_missing_count"] = df[EMBRYO_STAGE_COLS].isna().sum(axis=1)
    df["배아_이식_여부"] = (df["배아_stage_missing_count"] < len(EMBRYO_STAGE_COLS)).astype(int)
    
    # 4. 배아_진행_단계
    def embryo_stage(row):
        if row['배아_이식_여부'] == 0:
            return '배아단계_미도달'
        total = pd.to_numeric(row.get('총 생성 배아 수'), errors='coerce')
        implanted = pd.to_numeric(row.get('이식된 배아 수'), errors='coerce')
        if pd.isna(total) or total == 0:
            return '배아생성_실패'
        elif pd.isna(implanted) or implanted == 0:
            return '이식_미실시'
        else:
            return '이식_완료'
    
    df['배아_진행_단계'] = df.apply(embryo_stage, axis=1)
    
    # 5. 총시술_bin3
    def collapse_trials(x):
        if pd.isna(x): return "Unknown"
        if x == "0회": return "0회"
        if x in ["1회", "2회"]: return "1-2회"
        return "3회 이상"
    
    df["총시술_bin3"] = df["총 시술 횟수"].apply(collapse_trials)
    
    # 6. 나이_3구간
    def age_group_simple(age):
        if pd.isna(age) or age == "알 수 없음": return "Unknown"
        if age == "만18-34세": return "34세 이하"
        if age in ["만35-37세", "만38-39세"]: return "35-39세"
        return "40세 이상"
    
    df["나이_3구간"] = df["시술 당시 나이"].apply(age_group_simple)
    
    # 7. 이식배아_구간
    def embryo_count_bin(count):
        count = pd.to_numeric(count, errors='coerce')
        if pd.isna(count) or count == 0: return "0개"
        elif count <= 2: return "1-2개"
        else: return "3개 이상"
    
    df['이식배아_구간'] = df['이식된 배아 수'].apply(embryo_count_bin)
    
    # 8. Day5_이식_여부
    d = pd.to_numeric(df["배아 이식 경과일"], errors="coerce")
    df["Day5_이식_여부"] = (d == 5).astype(int)
    
    # 9. 불임원인_복잡도
    tmp = df[INFERTILITY_COLS].apply(pd.to_numeric, errors="coerce").fillna(0)
    df["불임_원인_개수"] = tmp.sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0: return 'None'
        elif count == 1: return 'Single'
        elif count == 2: return 'Double'
        else: return 'Multiple'
    
    df['불임원인_복잡도'] = df['불임_원인_개수'].apply(infertility_complexity)
    
    # 10. 배아_해동_실시_여부
    df['배아_해동_실시_여부'] = df['배아 해동 경과일'].notna().astype(int)
    
    # 숫자형 안전 변환
    num_cols = [
        "총 생성 배아 수", "이식된 배아 수", "저장된 배아 수",
        "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
        "수집된 신선 난자 수", "혼합된 난자 수",
        "배아 이식 경과일", "난자 채취 경과일", "난자 혼합 경과일"
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    
    # 11-13. 비율 변수 (03번 검증됨)
    df["배아_생성_효율"] = safe_div(
        df["총 생성 배아 수"].fillna(0),
        df["수집된 신선 난자 수"].fillna(0)
    )
    
    df["배아_이식_비율"] = safe_div(
        df["이식된 배아 수"].fillna(0),
        df["총 생성 배아 수"].fillna(0)
    )
    
    df["배아_저장_비율"] = safe_div(
        df["저장된 배아 수"].fillna(0),
        df["총 생성 배아 수"].fillna(0)
    )
    
    # 14-15. 교호작용 (03번 검증됨)
    df['나이×Day5'] = df['시술 당시 나이'].astype(str) + '_' + df['Day5_이식_여부'].astype(str)
    df['시술횟수×나이'] = df['총시술_bin3'].astype(str) + '_' + df['나이_3구간'].astype(str)
    
    # ===== 🆕 07번에서 효과 확인된 Feature (5개만!) =====
    
    # 16. 나이×배아수×Day5 (3-way, 중요도 0.0027)
    df['나이×배아수×Day5'] = (
        df['나이_3구간'].astype(str) + '_' +
        df['이식배아_구간'].astype(str) + '_' +
        df['Day5_이식_여부'].astype(str)
    )
    
    # 17. 시술횟수×나이×불임 (3-way, 중요도 0.0021)
    df['시술횟수×나이×불임'] = (
        df['총시술_bin3'].astype(str) + '_' +
        df['나이_3구간'].astype(str) + '_' +
        df['불임원인_복잡도'].astype(str)
    )
    
    # 18. 확정_실패_케이스 (Post-processing용)
    storage_keywords = ['난자 저장용', '기증용', '배아 저장용', '연구용']
    df['확정_실패_케이스'] = 0
    if '배아 생성 주요 이유' in df.columns:
        df['확정_실패_케이스'] = df['배아 생성 주요 이유'].isin(storage_keywords).astype(int)
    if '시술 당시 나이' in df.columns:
        df['확정_실패_케이스'] = df['확정_실패_케이스'] | (df['시술 당시 나이'] == '알 수 없음').astype(int)
    
    # 19. 배아_활용률 (중요도 0.0008)
    df['배아_활용률'] = safe_div(
        (df['이식된 배아 수'].fillna(0) + df['저장된 배아 수'].fillna(0)),
        df['총 생성 배아 수'].fillna(0)
    )
    
    # 20. IVF_과거_성공률 (중요도 0.0008)
    trials_map = {'0회': 0, '1회': 1, '2회': 2, '3회': 3, '4회': 4, '5회': 5, '6회 이상': 7}
    ivf_trials = df['IVF 시술 횟수'].map(trials_map).fillna(0)
    ivf_preg = df['IVF 임신 횟수'].map(trials_map).fillna(0)
    df['IVF_과거_성공률'] = safe_div(ivf_preg, ivf_trials)
    
    return df

print("\n전처리 중...")
train_df = preprocess(train)
test_df = preprocess(test)

print(f"✅ Train shape: {train_df.shape}")
print(f"✅ Test shape: {test_df.shape}")

# ID 제거
train_data = train_df.drop(columns=[ID_COL, '배아_stage_missing_count'])
test_data = test_df.drop(columns=[ID_COL, '배아_stage_missing_count'])

print(f"\n🎯 최종 Feature 개수: {len(train_data.columns) - 1}개")
print(f"   - 03번 검증: 15개")
print(f"   - 07번 추가: 5개")
print(f"   - 총: 20개 (최적화!)")

# ============================================================
# 2) AutoGluon 학습 (최적화 설정)
# ============================================================
print("\n" + "="*80)
print("🏆 AutoGluon Training - 1등 탈환 모드! 🏆")
print("="*80)
print("\n⏰ 예상 소요 시간: 4시간")
print("🎯 목표 CV AUC: 0.7425+")
print("🏆 목표 LB: 0.7430+ (현재 1등: 0.74209)")
print("\n학습 시작...\n")

predictor = TabularPredictor(
    label=TARGET_COL,
    eval_metric='roc_auc',
    problem_type='binary',
    path='../outputs/autogluon_08_final_strike'
)

predictor.fit(
    train_data=train_data,
    time_limit=14400,        # 4시간
    presets='best_quality',
    num_bag_folds=10,        # 안정성
    num_bag_sets=2,          # 다양성
    num_stack_levels=2,      # 깊은 스태킹
    hyperparameters='multimodal',  # 다양한 모델
    auto_stack=True,         # 자동 스태킹 최적화
    verbosity=2
)

# ============================================================
# 3) 결과 확인
# ============================================================
print("\n" + "="*80)
print("📊 AutoGluon Results")
print("="*80)

leaderboard = predictor.leaderboard(train_data, silent=True)
print("\n=== Model Leaderboard (Top 20) ===")
print(leaderboard.head(20).to_string())

best_model = leaderboard.iloc[0]['model']
best_score = leaderboard.iloc[0]['score_val']

print(f"\n🏆 Best Model: {best_model}")
print(f"🎯 Best CV AUC: {best_score:.6f}")

print(f"\n📊 성적표:")
print(f"   03번 (기본):     0.7407")
print(f"   07번 (35개):     0.7386")
print(f"   08번 (20개):     {best_score:.6f}")
print(f"   Improvement:    +{best_score - 0.7407:.6f}")

if best_score >= 0.7425:
    print("\n🎉🎉🎉 목표 달성! 1등 가능성 높음! 🎉🎉🎉")
elif best_score >= 0.7415:
    print("\n😊 좋은 성적! 상위권 확정!")
else:
    print("\n🤔 03번보다 높으면 제출 고려")

# ============================================================
# 4) 예측 및 Submission
# ============================================================
print("\n" + "="*80)
print("🎯 Generating Predictions")
print("="*80)

test_predictions = predictor.predict_proba(test_data)

if isinstance(test_predictions, pd.DataFrame):
    test_proba = test_predictions[1].values
else:
    test_proba = test_predictions

# 🔥 Post-processing: 확정 실패 케이스 강제 0
mask = test_df['확정_실패_케이스'] == 1
if mask.sum() > 0:
    print(f"\n⚠️  확정 실패 케이스 {mask.sum()}개 → 강제로 0.0 설정")
    test_proba[mask] = 0.0

print(f"\n예측값 통계:")
print(f"Min:  {test_proba.min():.6f}")
print(f"Max:  {test_proba.max():.6f}")
print(f"Mean: {test_proba.mean():.6f}")
print(f"Std:  {test_proba.std():.6f}")

# Submission
sub['probability'] = test_proba
output_file = f"../outputs/08_submission_final_strike_CV{best_score:.6f}.csv"
sub.to_csv(output_file, index=False)

print(f"\n✅ Submission saved!")
print(f"   File: {output_file}")

# 상세 정보 저장
leaderboard.to_csv("../outputs/08_leaderboard.csv", index=False)
print("✅ Leaderboard saved: ../outputs/08_leaderboard.csv")

try:
    importance = predictor.feature_importance(train_data)
    importance.to_csv("../outputs/08_feature_importance.csv")
    print("✅ Feature importance saved")
    print("\n=== Top 20 Features ===")
    print(importance.head(20).to_string())
except:
    print("⚠️ Feature importance not available")

# 요약
summary = f"""
{'='*80}
🏆 08번 - 1등 탈환 작전 결과 🏆
{'='*80}

Feature 전략: 검증된 20개만!
  - 03번 검증 Feature: 15개
  - 07번 효과 확인: 5개
  - 총 Feature: {len(train_data.columns) - 1}개

Best Model: {best_model}
CV AUC: {best_score:.6f}

성적 비교:
  03번 (기본):     0.7407 (LB: 0.742)
  07번 (35개):     0.7386
  08번 (20개):     {best_score:.6f}
  
Improvement from 03번: +{best_score - 0.7407:.6f}

{'='*80}
현재 순위:
  1등: 0.74209
  나:  0.74200 (5등)
  
08번 예상 LB: {best_score + 0.0013:.6f}
(03번은 CV 0.7407 → LB 0.742로 +0.0013 상승)

{'='*80}
"""

if best_score >= 0.7425:
    summary += "\n🎉🎉🎉 1등 탈환 가능성 높음! 바로 제출! 🎉🎉🎉\n"
elif best_score >= 0.7415:
    summary += "\n😊 상위권 확정! 제출 추천!\n"
elif best_score > 0.7407:
    summary += "\n✅ 03번보다 높음! 제출 고려!\n"
else:
    summary += "\n🤔 03번 유지 검토...\n"

summary += "="*80

print("\n" + summary)

with open("../outputs/08_summary.txt", "w", encoding='utf-8') as f:
    f.write(summary)

print("\n✅ Summary saved: ../outputs/08_summary.txt")
print("\n" + "="*80)
print("🔥 08번 COMPLETE! 🔥")
print("="*80)

/opt/anaconda3/envs/fertility_ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



전처리 중...


Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=10, num_bag_sets=2
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 14400 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: ../outputs/autogluon_08_final_strike/ds_sub_fit/sub_fit_ho.


✅ Train shape: (256351, 86)
✅ Test shape: (90067, 85)

🎯 최종 Feature 개수: 83개
   - 03번 검증: 15개
   - 07번 추가: 5개
   - 총: 20개 (최적화!)

🏆 AutoGluon Training - 1등 탈환 모드! 🏆

⏰ 예상 소요 시간: 4시간
🎯 목표 CV AUC: 0.7425+
🏆 목표 LB: 0.7430+ (현재 1등: 0.74209)

학습 시작...



2026-02-07 17:20:03,442	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 3621 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 10779 seconds.
Starting full fit now with num_stack_levels 2.
Beginning AutoGluon training ... Time limit = 10779s
AutoGluon will save models to "../outputs/autogluon_08_final_strike"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Mon Jul 14 11:30:29 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       7.19 GB / 16.00 GB (44.9%)
Disk Space Avail:   207.29 GB / 460.43 GB (45.0%)
Train Data Rows:    256351
Train Data Columns: 83
Label Column:       임신 성공 여부
Pr


📊 AutoGluon Results

=== Model Leaderboard (Top 20) ===
                    model  score_test  score_val eval_metric  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0    LightGBMLarge_BAG_L1    0.774141   0.738185     roc_auc        7.245909       5.740938    12.988045                 7.245909                5.740938          12.988045            1       True          6
1    LightGBMLarge_BAG_L2    0.760447   0.739224     roc_auc       60.654647      25.169178  4404.662633                 9.467964                7.036826          21.447542            2       True         13
2    LightGBMLarge_BAG_L3    0.760427   0.739197     roc_auc      142.934995      46.904182  6664.697865                 9.285333                6.968925          20.498347            3       True         20
3       LightGBMXT_BAG_L3    0.757112   0.739111     roc_auc      139.452085      43.644873  6658.161908       

These features in provided data are not utilized by the predictor and will be ignored: ['불임 원인 - 여성 요인', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아_이식_여부']



⚠️  확정 실패 케이스 4205개 → 강제로 0.0 설정

예측값 통계:
Min:  0.000000
Max:  0.755408
Mean: 0.258951
Std:  0.159878

✅ Submission saved!
   File: ../outputs/08_submission_final_strike_CV0.738185.csv
✅ Leaderboard saved: ../outputs/08_leaderboard.csv


Computing feature importance via permutation shuffling for 77 features using 5000 rows with 5 shuffle sets...
	599.62s	= Expected runtime (119.92s per shuffle set)
	390.53s	= Actual runtime (Completed 5 of 5 shuffle sets)


✅ Feature importance saved

=== Top 20 Features ===
               importance    stddev   p_value  n  p99_high   p99_low
이식된 배아 수         0.020642  0.004170  0.000189  5  0.029228  0.012056
이식배아_구간          0.014499  0.003663  0.000450  5  0.022041  0.006957
시술 당시 나이         0.010428  0.003617  0.001490  5  0.017876  0.002980
저장된 배아 수         0.005230  0.003422  0.013420  5  0.012276 -0.001816
배아 이식 경과일        0.004973  0.001811  0.001783  5  0.008701  0.001245
나이×배아수×Day5      0.004520  0.002539  0.008201  5  0.009749 -0.000709
시술 시기 코드         0.003609  0.000621  0.000101  5  0.004887  0.002330
나이_3구간           0.003155  0.002089  0.013936  5  0.007456 -0.001147
배아_진행_단계         0.003114  0.001639  0.006584  5  0.006487 -0.000260
배아_생성_효율         0.002937  0.000625  0.000232  5  0.004224  0.001650
나이×Day5          0.002922  0.001354  0.004244  5  0.005709  0.000134
수집된 신선 난자 수      0.002558  0.000840  0.001215  5  0.004287  0.000828
시술횟수×나이×불임       0.002544  0.000197  0.000004  5  0